# Library Load

In [1]:
# !pip install efficientnet_pytorch
# !pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git

In [1]:
import torch
import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
from torch_poly_lr_decay import PolynomialLRDecay
import random


torch.set_num_threads(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Dacon Dataset Load

In [2]:
labels_df = pd.read_csv('dataset/dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('dataset/dirty_mnist_2nd/*')))[:]
labels = np.array(labels_df.values[:,1:])

In [3]:
test_imgs_dir = np.array(sorted(glob.glob('dataset/test_dirty_mnist_2nd/*')))

In [4]:
imgs=[]
for path in tqdm(imgs_dir[:]):
    img=cv2.imread(path, cv2.IMREAD_COLOR)
    imgs.append(img)
imgs=np.array(imgs)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:13<00:00, 679.42it/s]


In [5]:
# 저장소에서 load
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass
    


# 메모리에서 load
class MnistDataset_v2(Dataset):
    def __init__(self, imgs=None, labels=None, transform=None, train=True):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.train=train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get1
        img = self.imgs[idx]
        img = self.transform(img)
        
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
    


# reproduction을 위한 seed 설정

In [6]:
# https://dacon.io/competitions/official/235697/codeshare/2363?page=1&dtype=recent&ptype=pub
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  

# model 정의

In [7]:
# EfficientNet -b0(pretrained)
# MultiLabel output

class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels):
        super(EfficientNet_MultiLabel, self).__init__()
        self.network = EfficientNet.from_pretrained('efficientnet-b0', in_channels=in_channels)
        self.output_layer = nn.Linear(1000, 26)

    def forward(self, x):
        x = F.relu(self.network(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# 데이터 분리
- 해당 코드에서는 1fold만 실행합니다.
- 모두 실행하려면 훈련시 반복횟수를 5로 바꾸어주면 됩니다.

In [8]:
# 해당 코드에서는 1fold만 실행

kf = KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(imgs):
    folds.append((train_idx, valid_idx))

In [10]:
### seed_everything(42)

# 5개의 fold 모두 실행하려면 for문을 5번 돌리면 됩니다.
for fold in range(1):
    model = EfficientNet_MultiLabel(in_channels=3).to(device)
#     model = nn.DataParallel(model)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]



    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()
        ])
    valid_transform = transforms.Compose([
        transforms.ToTensor(),
        ])


    epochs=30
    batch_size=47         # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.
    
    
    
    # Data Loader
    train_dataset = MnistDataset_v2(imgs = imgs[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MnistDataset_v2(imgs = imgs[valid_idx], labels = labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)       
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    # 
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    decay_steps = (len(train_dataset)//batch_size)*epochs
    scheduler_poly_lr_decay = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)

    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0
    for epoch in range(epochs):
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_loader)):
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            scheduler_poly_lr_decay.step()

            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []
        with torch.no_grad():
            for n_valid, (X_valid, y_valid) in enumerate((valid_loader)):
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))
            
        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            torch.save(model.state_dict(), 'model/EfficientNetB0-fold{}.pt'.format(fold))
            valid_best_accuracy = np.mean(valid_batch_accuracy)
        print('fold : {}\tepoch : {:02d}\ttrain_accuracy / loss : {:.5f} / {:.5f}\tvalid_accuracy / loss : {:.5f} / {:.5f}\ttime : {:.0f}'.format(fold+1, epoch+1,
                                                                                                                                              np.mean(batch_accuracy_list),
                                                                                                                                              np.mean(batch_loss_list),
                                                                                                                                              np.mean(valid_batch_accuracy), 
                                                                                                                                              np.mean(valid_batch_loss),
                                                                                                                                              time.time()-start))

Loaded pretrained weights for efficientnet-b0


C:\Users\JeongMyeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTe

fold : 1	epoch : 01	train_accuracy / loss : 0.57832 / 0.67041	valid_accuracy / loss : 0.60466 / 0.66580	time : 362
fold : 1	epoch : 02	train_accuracy / loss : 0.65314 / 0.61597	valid_accuracy / loss : 0.66300 / 0.62072	time : 362
fold : 1	epoch : 03	train_accuracy / loss : 0.70415 / 0.56392	valid_accuracy / loss : 0.71476 / 0.55302	time : 352
fold : 1	epoch : 04	train_accuracy / loss : 0.74050 / 0.51942	valid_accuracy / loss : 0.74490 / 0.51546	time : 351
fold : 1	epoch : 05	train_accuracy / loss : 0.76704 / 0.48314	valid_accuracy / loss : 0.76168 / 0.49233	time : 352
fold : 1	epoch : 06	train_accuracy / loss : 0.78603 / 0.45380	valid_accuracy / loss : 0.78428 / 0.45881	time : 351
fold : 1	epoch : 07	train_accuracy / loss : 0.80202 / 0.42815	valid_accuracy / loss : 0.79765 / 0.43725	time : 350
fold : 1	epoch : 08	train_accuracy / loss : 0.81593 / 0.40455	valid_accuracy / loss : 0.80026 / 0.43284	time : 351
fold : 1	epoch : 09	train_accuracy / loss : 0.82878 / 0.38237	valid_accuracy / l

# Test Image 로드

In [9]:
test_imgs=[]
for path in tqdm(test_imgs_dir):
    test_img=cv2.imread(path, cv2.IMREAD_COLOR)
    test_imgs.append(test_img)
test_imgs=np.array(test_imgs)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:07<00:00, 645.88it/s]


In [10]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        ])

# Test 추론


In [11]:
submission = pd.read_csv('dataset/sample_submission.csv')

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('model/EfficientNetB0-fold{}.pt'.format(fold)))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission.iloc[n*32:(n+1)*32,1:] += pred_test

Loaded pretrained weights for efficientnet-b0


  0%|                                                                                          | 0/157 [00:00<?, ?it/s]C:\Users\JeongMyeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:13<00:00, 12.01it/s]


# 제출물 생성

In [12]:
submission.iloc[:,1:] = np.where(submission.values[:,1:]>=0.5, 1,0)

In [13]:
submission.to_csv('EfficientNetB0-fold0.csv', index=False)